# Project name: Customer Segmentation for E-commerce

Category:  Machine Learning, Customer Analytics

Importance: Identify customer segments based on behavior for targeted marketing and personalization.

Levels:
Level 1: Basic segmentation using RFM analysis.

Level 2: Incorporate machine learning clustering algorithms.

Level 3: Compairing RFM with clustering algorithms.

# Challenges

1. Should have to know RFM
2. Rules to segmantation without ML
3. K means clustering 

# RFM

1. Recency (R): Refers to the time since a customer's last purchase. It measures how recently a customer has interacted or made a purchase. The assumption is that more recent customers are likely to be more engaged and responsive.

2. Frequency (F): Represents the number of times a customer has made a purchase within a given period. It indicates the level of engagement and loyalty. Higher frequency often suggests a more loyal customer.

3. Monetary Value (M): Reflects the total amount of money a customer has spent on purchases. It measures the customer's contribution to the business in terms of revenue. Higher monetary value indicates customers who are more valuable to the business.

Import libraries for reading, preprocessing and vizualization 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import missingno as msno
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = r'G:\Final project24\Customer Segmentation for E-commerce\Sales Data.xlsx'
df = pd.read_excel(path)
df.head()

# Data preprocessing

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize=(8,5))

msno.matrix(df)
plt.title('Missing value plot')

plt.show()

In [ ]:
# Find percentage wise null
null_count = df.isnull().sum()
null_percentage = ((df.isnull().sum()/len(df))*100).round(2)

result_df = pd.DataFrame({"Null counts": null_count[null_count>0],"null percentage": null_percentage[null_count>0]})

print(result_df)

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print(col)
    print(df[col].value_counts())
    print("-" * 100)

In [ ]:
for col in df.describe(include = 'object').columns:
    print(col)
    print(df[col].unique())
    print("-"*100)

# EDA 

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['Order Date'], df['Sales'], alpha=0.5)

# Set labels and title
plt.xlabel('Order Date')
plt.ylabel('Sales')
plt.title('Scatter Plot of Sales Over Time')

# Show the plot
plt.show()

In [ ]:
# Sort the DataFrame by 'Order Date' for a chronological line plot
#df = df.sort_values('Order Date')

# Line plot
plt.figure(figsize=(10, 6))
plt.plot(df['Order Date'], df['Sales'], marker='o', linestyle='-', color='g')

# Set labels and title
plt.xlabel('Order Date')
plt.ylabel('Sales')
plt.title('Line Plot of Sales Over Time')

# Show the plot
plt.show()

In [ ]:
df.describe()

In [ ]:
# Find outliers

plt.figure(figsize=(10, 3))

sns.boxplot(x=df["Sales"])
    
plt.tight_layout()
plt.show()

In [ ]:
# Capping outliers using the IQR method

Q1 = df['Sales'].quantile(0.25)
Q3 = df['Sales'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds to cap outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Cap outliers by setting values beyond bounds to the bounds
df['Sales'] = df['Sales'].clip(lower=lower_bound, upper=upper_bound)

# Display the box plots after capping outliers
plt.figure(figsize=(10, 3))
sns.boxplot(x=df['Sales'])

plt.tight_layout()
plt.show()

In [ ]:
df['Sales'].skew()

In [ ]:
from scipy.stats import boxcox

# Applying the Box-Cox transformation
df['Sales'], lambda_value = boxcox(df['Sales'] + 1)  # Adding 1 to handle zero values

# Checking the skewness after transformation
print("Skewness after transformation:", df['Sales'].skew())

# Plotting the distribution before and after transformation
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['Sales'], kde=True)
plt.title('Before Transformation')

plt.subplot(1, 2, 2)
sns.histplot(df['Sales'], kde=True)
plt.title('After Box-Cox Transformation')

plt.show()

# Print the lambda value
print("Optimal lambda value:", lambda_value)

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [ ]:
df.head()

In [ ]:
df['Customer ID'].unique()

# RFM Anallysis with python

Calculating recency, frequency and monetry

In [ ]:
# Calculate recency based on 'Order Date'
today = pd.to_datetime('2023-12-01')
df['Recency'] = (today - df['Order Date']).dt.days

# Calculate frequency
df['Frequency'] = df.groupby('Customer ID')['Order Date'].transform('count')

# Calculate monetary
df['Monetary'] = df.groupby('Customer ID')['Sales'].transform('sum')

df.head()

In [ ]:
df.drop(columns=['Order ID','Product ID'], inplace = True)

In [ ]:
# Convert 'Order Date' to datetime if not already
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Group by 'Customer ID' and aggregate the values
agg_df = df.groupby('Customer ID').agg({
    'Order Date': 'first',
    'Customer Name': 'first',
    'Sales': 'sum',
    'Recency': 'sum',  # You can use mean or any other aggregation function depending on your requirement
    'Frequency': 'sum',  # Assuming you want the maximum frequency
    'Monetary': 'sum',
}).reset_index()

# Print the aggregated DataFrame
agg_df

Create bin

In [ ]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
agg_df['RecencyScore'] = pd.cut(agg_df['Recency'], bins=5, labels=recency_scores)
agg_df['FrequencyScore'] = pd.cut(agg_df['Frequency'], bins=5, labels=frequency_scores)
agg_df['MonetaryScore'] = pd.cut(agg_df['Monetary'], bins=5, labels=monetary_scores)

# Convert RFM scores to numeric type
agg_df['RecencyScore'] = agg_df['RecencyScore'].astype(int)
agg_df['FrequencyScore'] = agg_df['FrequencyScore'].astype(int)
agg_df['MonetaryScore'] = agg_df['MonetaryScore'].astype(int)

# Calculate RFM score by combining the individual scores
agg_df['RFM_Score'] = agg_df['RecencyScore'] + agg_df['FrequencyScore'] + agg_df['MonetaryScore']

In [ ]:
# List of columns for which you want to create box plots
columns_to_plot = ['Sales', 'Recency', 'Frequency', 'Monetary',
       'RecencyScore', 'FrequencyScore', 'RFM_Score']

# Set the size of the plot
plt.figure(figsize=(20, 15))

# Loop through the columns and create box plots
for i, column in enumerate(columns_to_plot, 1):
    plt.subplot(4, 2, i)  # Adjust the subplot grid as needed
    sns.boxplot(x=agg_df[column])
    plt.title(f'Box Plot for {column}')

plt.tight_layout()
plt.show()

In [ ]:
# List of columns for which you want to create box plots
columns_to_cap = ['Sales', 'Recency', 'Frequency', 'Monetary','RecencyScore', 'FrequencyScore', 'RFM_Score']

# Capping outliers using the IQR method

for column in columns_to_cap:
    Q1 = agg_df[column].quantile(0.25)
    Q3 = agg_df[column].quantile(0.75)
    IQR = Q3 - Q1
    # Define the lower and upper bounds to cap outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Cap outliers by setting values beyond bounds to the bounds
    agg_df[column] = agg_df[column].clip(lower=lower_bound, upper=upper_bound)

# Display the box plots after capping outliers
plt.figure(figsize=(20, 15))

# Loop through the columns and create box plots
for i, column in enumerate(columns_to_cap, 1):
    plt.subplot(4, 2, i)  # Adjust the subplot grid as needed
    sns.boxplot(x=agg_df[column])
    plt.title(f'Box Plot for {column}')

plt.tight_layout()
plt.show()

In [ ]:
agg_df.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame and numeric_agg_df contains the numeric columns
numeric_agg_df = ['Sales', 'Recency','Frequency', 'Monetary', 'RecencyScore',
                 'MonetaryScore', 'RFM_Score']
corr_mat = agg_df[numeric_agg_df].corr()

sns.heatmap(corr_mat, annot=True, fmt=".2f", cmap="coolwarm", linewidth = 0.5)
plt.show()

In [ ]:
agg_df['RFM_Score'].unique()

In [ ]:
# Create a new column for RFM Customer Segments
agg_df['RFM Customer Segments'] = ''

# Assign RFM segments based on RFM score thresholds
agg_df.loc[agg_df['RFM_Score'] >= 8, 'RFM Customer Segments'] = 'Top Customer'
agg_df.loc[(agg_df['RFM_Score'] >= 7) & (agg_df['RFM_Score'] < 8), 'RFM Customer Segments'] = 'High Value Customer'
agg_df.loc[(agg_df['RFM_Score'] >= 6) & (agg_df['RFM_Score'] < 7), 'RFM Customer Segments'] = 'Low Value Customer'
# agg_df.loc[(agg_df['RFM_Score'] >= 6) & (agg_df['RFM_Score'] < 7), 'RFM Customer Segments'] = "Low Value Customer"
agg_df.loc[(agg_df['RFM_Score'] < 6), 'RFM Customer Segments'] = "Lost Customer"

In [ ]:
agg_df.head()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Assuming you have three dimensions: Recency, Frequency, and Monetary
# Assuming 'Monetary' is the hue column
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with different colors based on the 'Monetary' column
scatter = ax.scatter(
    agg_df['Recency'], 
    agg_df['Frequency'], 
    agg_df['Monetary'], 
    c=agg_df['RFM_Score'],  # Use 'Monetary' column as the hue
    cmap='viridis', 
    s=50
)

# Set labels for each axis
ax.set_xlabel('RecencyScore')
ax.set_ylabel('FrequencyScore')
ax.set_zlabel('MonetaryScore')

# Add a colorbar
colorbar = plt.colorbar(scatter, ax=ax)
colorbar.set_label('Monetary')

# Show the plot
plt.show()

In [ ]:
agg_df.shape

# Show the Results

In [ ]:
# Assuming 'input_customer_id' is the ID you want to input
input_customer_id = 'SO-20335'  # Replace with the actual customer ID

# Filter the DataFrame based on the input customer ID
selected_customer = agg_df[agg_df['Customer ID'] == input_customer_id]

# Display all details for the selected customer
print("Details for Customer ID:", input_customer_id)
selected_customer

In [ ]:
# Filter the DataFrame to get only the rows where 'RFM Customer Segments' is 'Top Customer'
top_customer_df = agg_df[agg_df['RFM Customer Segments'] == 'Top Customer']

# Display the Customer IDs of the top customers
top_customer_df_ids = top_customer_df['Customer ID'].unique()
print("top_Customer_Ids:", len(top_customer_df_ids))

In [ ]:
# Filter the DataFrame to get only the rows where 'RFM Customer Segments' is 'Top Customer'
high_value_customers = agg_df[agg_df['RFM Customer Segments'] == 'High Value Customer']

# Display the Customer IDs of the top customers
high_value_customer_ids = high_value_customers['Customer ID'].unique()
print("High_Value_Customer_Ids:", len(high_value_customer_ids))

In [ ]:
# Filter the DataFrame to get only the rows where 'RFM Customer Segments' is 'Top Customer'
#medium_value_customers = agg_df[agg_df['RFM Customer Segments'] == 'Medium Value Customer']

# Display the Customer IDs of the top customers
#medium_value_customer_ids = medium_value_customers['Customer ID'].unique()
#print("Medium_Value_Customer_IDs:", len(medium_value_customer_ids))

In [ ]:
# Filter the DataFrame to get only the rows where 'RFM Customer Segments' is 'Top Customer'
low_value_customers = agg_df[agg_df['RFM Customer Segments'] == 'Low Value Customer']

# Display the Customer IDs of the top customers
low_value_customer_ids = low_value_customers['Customer ID'].unique()
print("Low_Value_Customer_IDs:", len(low_value_customer_ids))

In [ ]:
# Filter the DataFrame to get only the rows where 'RFM Customer Segments' is 'Top Customer'
lost_customers = agg_df[agg_df['RFM Customer Segments'] == 'Lost Customer']

# Display the Customer IDs of the top customers
lost_customer_ids = lost_customers['Customer ID'].unique()
print("Lost_Customer_IDs:", len(lost_customer_ids))

# ML

In [ ]:
from scipy import stats

from sklearn.cluster import KMeans

columns_of_interest = ['Sales', 'Recency', 'Frequency', 'Monetary',
       'RecencyScore', 'FrequencyScore', 'RFM_Score']

for column in columns_of_interest:
    if column == 'Id':
        continue  # Skip the 'Id' column
    skewness = agg_df[column].skew()
    if skewness > 0:
        skew_type = "Right_skewed"
    elif skewness < 0:
        skew_type = "Left_skewed"
    else:
        skew_type = "Approximately normal"
    print(f"Column={column}: Skewness={round(skewness, 2)}, Distribution={skew_type}")

In [ ]:
from scipy.stats import boxcox

# Specify the columns with right-skewed distributions
right_skewed_columns = ['Sales', 'Recency', 'Frequency', 'Monetary','RecencyScore','RFM_Score']

# Apply Box-Cox transformation to each right-skewed column
for column in right_skewed_columns:
    # Add a small constant to avoid issues with zero values
    agg_df[column] = agg_df[column] + 1
    transformed_data, lambda_value = boxcox(agg_df[column])
    agg_df[column] = transformed_data

# Visualize the transformed data
plt.figure(figsize=(15, 8))
for i, column in enumerate(right_skewed_columns, 1):
    plt.subplot(3, 2, i)
    sns.histplot(agg_df[column], kde=True)
    plt.title(f'Box-Cox Transformed {column}')

plt.tight_layout()
plt.show()

In [ ]:
# from sklearn.preprocessing import StandardScalar

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# Extract RFM columns
rfm_data = agg_df[['RecencyScore', 'FrequencyScore', 'MonetaryScore']]

# Standardize the data
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_data)

# Elbow

In [ ]:
# Determine the optimal number of clusters using the Elbow Method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(rfm_scaled)
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method graph with different colors
fig, ax = plt.subplots()
ax.plot(range(1, 11), wcss, marker='o', linestyle='-', color='b', label='WCSS')

# Highlight the optimal number of clusters with a different color
optimal_clusters = 4  # Change this based on your analysis
optimal_color = 'r'
ax.scatter(optimal_clusters, wcss[optimal_clusters - 1], c=optimal_color, s=200, label='Optimal k')

ax.set_title('Elbow Method for Optimal k')
ax.set_xlabel('Number of clusters')
ax.set_ylabel('WCSS')
ax.legend()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

# Assuming X is your scaled data
X = agg_df[['RecencyScore', 'FrequencyScore', 'MonetaryScore']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Try different values of K
for k in range(2, 6):
    kmeans = KMeans(n_clusters=k, random_state=0)
    agg_df['KMeans_Cluster'] = kmeans.fit_predict(X_scaled)
    
    # Evaluate clustering using silhouette score
    silhouette_avg = silhouette_score(X_scaled, agg_df['KMeans_Cluster'])
    
    print(f"For K={k}, Silhouette Score: {silhouette_avg}")

In [ ]:
# Apply k-means clustering
X = agg_df[['RecencyScore', 'FrequencyScore', 'MonetaryScore']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=0)
agg_df['KMeans_Cluster'] = kmeans.fit_predict(X_scaled)

In [ ]:
agg_df.head()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming you have three dimensions: Recency, Frequency, and Monetary
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for the clusters
scatter = ax.scatter(agg_df['Recency'], agg_df['Frequency'], agg_df['Monetary'], c=agg_df['KMeans_Cluster'], cmap='viridis', s=50)

# Set labels for each axis
ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary')

# Create a custom legend
legend_labels = agg_df['KMeans_Cluster'].unique()
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=f'Cluster {label}', markerfacecolor=scatter.to_rgba(label)) for label in legend_labels]
ax.legend(handles=legend_handles, title='Clusters')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming you have three dimensions: Recency, Frequency, and Monetary
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for the clusters
scatter = ax.scatter(agg_df['RecencyScore'], agg_df['FrequencyScore'], agg_df['MonetaryScore'],
                     c=agg_df['KMeans_Cluster'], cmap='viridis', s=50)

# Set labels for each axis
ax.set_xlabel('RecencyScore')
ax.set_ylabel('FrequencyScore')
ax.set_zlabel('MonetaryScore')

# Create a custom legend
legend_labels = agg_df['KMeans_Cluster'].unique()
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=f'Cluster {label}', markerfacecolor=scatter.to_rgba(label)) for label in legend_labels]
ax.legend(handles=legend_handles, title='Clusters')

# Show the plot
plt.show()

In [ ]:
import pandas as pd

# Assuming you have already applied KMeans clustering and have the 'KMeans_Cluster' column in your DataFrame
# Assuming you have already calculated RFM scores and have the 'RFM Customer Segments' column in your DataFrame

# Create a DataFrame for cluster-wise IDs
cluster_category = pd.DataFrame()
for cluster_id in agg_df['KMeans_Cluster'].unique():
    cluster_data = agg_df[agg_df['KMeans_Cluster'] == cluster_id]
    cluster_category[f'Cluster_{cluster_id}_IDs'] = cluster_data['Customer ID']

# Create a DataFrame for RFM score-wise IDs
rfm_category = pd.DataFrame()
for rfm_cat in agg_df['RFM Customer Segments'].unique():
    rfm_data = agg_df[agg_df['RFM Customer Segments'] == rfm_cat]
    rfm_category[f'RFM_{rfm_cat}_IDs'] = rfm_data['Customer ID']

# Combine both DataFrames
combined_ids = pd.DataFrame({
    'Customer ID': agg_df['Customer ID'],
    'Cluster_category': agg_df['KMeans_Cluster'],
    'RFM_category': agg_df['RFM Customer Segments']
})

# Display the combined DataFrame
combined_ids

In [ ]:
from sklearn.metrics import adjusted_rand_score

# Assuming you have the true labels for comparison (e.g., true RFM segmentation)
true_labels = agg_df['RFM Customer Segments']

# Calculate Adjusted Rand Index
ari_score = adjusted_rand_score(true_labels, agg_df['KMeans_Cluster'])

print(f'Adjusted Rand Index between KMeans clustering and RFM segmentation: {ari_score}')

In [ ]:
from sklearn.metrics import adjusted_mutual_info_score

# Assuming you have the true labels for comparison (e.g., true RFM segmentation)
true_labels = agg_df['RFM Customer Segments']

# Calculate Adjusted Mutual Information
ami_score = adjusted_mutual_info_score(true_labels, agg_df['KMeans_Cluster'])

print(f'Adjusted Mutual Information between KMeans clustering and RFM segmentation: {ami_score}')

In [ ]:
agg_df.to_csv("test_rfm.csv", index = False)

In [ ]:
# Save the combined DataFrame to a CSV file
combined_ids.to_csv('combined_ids.csv', index=False)